## Astronomical image processing -- detecting stars, aligning images, and making color images

Following on last lecture, we're going to use some of the things we have learned -- matched filtering, estimating "background" levels, and manipulating RGB images -- to build an astronomical mosaic image from raw image data.

Unlike last week, I'm going to install and use the `FITSIO` package to handle real astronomy data.

In [ ]:
] add Images ImageIO ImageFiltering Plots FITSIO ImageShow StatsBase

We are going to read a set of images from the Sloan Digital Sky Survey (SDSS).

SDSS used a special camera to do "drift scanning" of the sky.  In drift scanning, the telescope is kept pointed almost stationary, and the sky is observed as the Earth rotates (causing the sky to drift past the camera).  The CCD chips in the camera are read out at exactly the same speed as the sky drifts past.
    

In [ ]:
`bunzip2

In [ ]:
using Images, ImageIO

In [ ]:
load("sdss-camera.jpg")

Above is a photograph of the SDSS camera.  It has 6 columns of chips (CCDs), with 5 chips in each column.  Each chip has a bandpass filter permanently mounted to it.  When drift scanning, a star would first appear at the top of the camera and drift down to the bottom.  The chips are perfectly rotated so that a star will drift exactly down the columns of the CCD.

The five filters are called "u", "g", "r", "i", and "z":
"u" is near-ultraviolet
"g" is "green"
"r" is "red"
"i" is near-infrared
"z" is further into the infrared

Top to bottom, the filter order is "r","i","u","z","g".  In the photo above, you can see that the top row of chips has a red filter, and the bottom row has a bluish filter, while the middle three look black to our eyes.

For typical stars, the "g", "r", and "i" filters are most sensitive, and these are the ones we will use today.

In [ ]:
# Astronomical images are almost always in FITS = Flexible Image Transport System format
using FITSIO

In [ ]:
# I downloaded calibrated data from the Sloan Digital Sky Survey web site
# -- https://dr12.sdss.org/fields/runCamcolField?run=752&camcol=5&field=408

# Let's read the "r"-band image.
rfile = FITS("frame-r-000752-5-0408.fits")
rimage = read(rfile[1])
size(rimage), typeof(rimage)

In [ ]:
# FITS files typically come with a lot of "header" information
read_header(rfile[1]);

In [ ]:
# Let's have a look!
Gray.(rimage)

In [ ]:
# That image is a little big to handle -- for convenience let's cut out a sub-image to work with interactively.
# This range "just happens" to have two nice galaxies in it!
rsub = rimage[1000:1500, 500:1000];
Gray.(rsub)

Let's investigate the properties of this image.  We'll start by histogramming the pixel values.

In [ ]:
using Plots

In [ ]:
histogram(vec(rimage))

As I claimed last week, astronomical images often have a large dynamic range -- so this histogram isn't super useful because the histogram is auto-stretching to show a few pixels with very large values.  Looking in log space (vertically), we can see that more clearly.

In [ ]:
histogram(vec(rimage), yscale=:log10)

Let's check out whether we need to do a "background" or "sky" subtraction, by zooming in to the peak of the histogram.  We'll measure where that is using the `quantile` trick from last week.

In [ ]:
using Statistics

In [ ]:
lo,mid,hi = quantile(vec(rimage), [0.16, 0.5, 0.84])

In [ ]:
histogram(vec(rimage), bins=range(-0.1, 1, length=50))

Okay, so it looks like this image has already had a background level subtracted in calibration.  That's convenient!

Next, let's load images from the other filters observed by this camera.

In [ ]:
gfile = FITS("frame-g-000752-5-0408.fits")
gimage = read(gfile[1])
ifile = FITS("frame-i-000752-5-0408.fits")
iimage = read(ifile[1])
# We'll take the same sub-images for looking at.
gsub = gimage[1000:1500, 500:1000];
isub = iimage[1000:1500, 500:1000];
[Gray.(gsub) Gray.(rsub) Gray.(isub)]

Let's try making a color image out of these three images.

In [ ]:
colorview(RGB, isub, rsub, gsub)

Yuck!  It looks like these images are not pixel-by-pixel aligned.

Now, we could probably eyeball the offsets between the image layers, but I am going to show you a more
sophisticated approach (that generalizes to rotations, and is similar to what I have done in my research work
for aligning a large Hubble Space Telescope survey): we are going to detect stars in the different images,
and find an offset that aligns them!

Detecting stars in astronomical images is a *perfect* setting for using a *matched filter*.

We know that the stars are tiny dots (nearly "point sources"), which get blurred out by the atmosphere.  We can approximate that atmospheric blurring as a Gaussian.  The width of that Gaussian is measured and reported by the SDSS data-processing pipeline.

In [ ]:
# From the web page for this image: https://dr12.sdss.org/fields/runCamcolField?run=752&camcol=5&field=408
# psf_fwhm: ~1.3
# That's in arcseconds FWHM, and the pixel scale of the SDSS camera is 0.396"/pixel, so:
# 1.3" / 0.396" = 3.3 pixels FWHM / 2.35 = 1.4 pixels Gaussian sigma.

In [ ]:
using ImageFiltering

In [ ]:
sigma = 1.4
# Construct the filtering kernel:
gauss_kernel = Kernel.gaussian(sigma)
# Apply the filter!
rfilt = imfilter(rsub, gauss_kernel)

# What does that filter do to the noise?
# First estimate the noise in the original image:
lo,mid,hi = quantile(vec(rimage), [0.16, 0.5, 0.84])
rsigma = mid - lo
# Using Gaussian statistics, you can compute the noise of the filtered image:
rfilt_sigma = rsigma / (2. * sqrt(pi) * sigma)

# Let's look at the filtered image
Gray.(rfilt)

In [ ]:
rfilt_sigma

In [ ]:
histogram(vec(rfilt), bins=range(-5. * rfilt_sigma, 10. * rfilt_sigma, length=50))

Next, we're going to search for *peak* in the filtered image, above N sigma.

In [ ]:
function find_peaks(filt, filt_sigma, nsigma)
    H,W = size(filt)
    peak_x = []
    peak_y = []
    for i = 2:H-1
        for j = 2:W-1
            if ((filt[i, j] > (nsigma * filt_sigma)) &&
                (filt[i, j] > filt[i-1, j-1]) &&
                (filt[i, j] > filt[i-1, j  ]) &&
                (filt[i, j] > filt[i-1, j+1]) &&
                (filt[i, j] > filt[i+1, j-1]) &&
                (filt[i, j] > filt[i+1, j  ]) &&
                (filt[i, j] > filt[i+1, j+1]) &&
                (filt[i, j] > filt[i  , j-1]) &&
                (filt[i, j] > filt[i  , j+1]))
                append!(peak_x, j)
                append!(peak_y, i)
            end
        end
    end
    peak_x, peak_y
end

In [ ]:
nsigma = 8
rx,ry = find_peaks(rfilt, rfilt_sigma, nsigma);

In [ ]:
plot(10. * Gray.(rsub))
plot!(rx, ry, seriestype = :scatter, markershape=:x, color="red")
    #markersize=5, markerstrokecolor="red", 
    #markercolor = :white)
    #markeralpha=0)

In [ ]:
# Repeat for "g" and "i" bands!
lo,mid,hi = quantile(vec(gimage), [0.16, 0.5, 0.84])
gsigma = mid - lo
gfilt_sigma = gsigma / (2. * sqrt(pi) * sigma)
gfilt = imfilter(gsub, gauss_kernel)
gx,gy = find_peaks(gfilt, gfilt_sigma, nsigma);
plot(10. * Gray.(gsub))
plot!(gx, gy, seriestype = :scatter, markershape=:x, color="red")

In [ ]:
lo,mid,hi = quantile(vec(iimage), [0.16, 0.5, 0.84])
isigma = mid - lo
ifilt_sigma = isigma / (2. * sqrt(pi) * sigma)
ifilt = imfilter(isub, gauss_kernel)
ix,iy = find_peaks(ifilt, ifilt_sigma, nsigma);
plot(10. * Gray.(isub))
plot!(ix, iy, seriestype = :scatter, markershape=:x, color="red")

Now, we're going to take our detected star positions and search for an offset that causes many of the star positions to line up!

Start by finding nearby pairs of stars, and then we'll histogram their delta-x, delta-y offsets.

In [ ]:
function find_nearby(x1,y1, x2,y2, dist)
    I = []
    J = []
    for i in 1:length(x1)
        for j in 1:length(x2)
            if abs(x1[i]-x2[j]) + abs(y1[i]-y2[j]) < dist
                append!(I, i)
                append!(J, j)
            end
        end
    end
    I,J
end

In [ ]:
I,J = find_nearby(rx,ry, gx,gy, 50.)

In [ ]:
dx = rx[I] - gx[J]
dy = ry[I] - gy[J];

In [ ]:
plot(dx, dy, seriestype=:scatter, marker=:x, alpha=0.5)

In [ ]:
histogram2d(dx, dy, bins=20)

In [ ]:
using StatsBase

We want to *find* the peak of that histogram.  This is the (rather ugly) way I found to do that:

In [ ]:
function find_most_common(dx, dy)
    H = fit(Histogram, (dx, dy), (minimum(dx):maximum(dx), minimum(dy):maximum(dy)))
    a = argmax(H.weights)
    ax,ay = a.I
    xe,ye = H.edges
    shiftx, shifty = xe[ax], ye[ay]
end

In [ ]:
shiftx,shifty = find_most_common(dx,dy)

Okay, we have found an integer pixel offset between the "g" and "r" band images!  Let's apply that shift to the images.

In [ ]:
colorview(RGB, isub, rsub, gsub)

In [ ]:
H,W = size(rsub)
zsub = zeros(H,W)
colorview(RGB, zsub, rsub, gsub)

We'll apply that shift by creating *padded* images, and then copying our images into the padded arrays, with offsets.

In [ ]:
padding = 20
H,W = size(rsub)
padr = zeros((H + 2*padding, W + 2*padding))
padr[padding+1:padding+H, padding+1:padding+W] = rsub;

padg = zeros((H + 2*padding, W + 2*padding))
padg[shifty + padding+1 : shifty + padding+H, shiftx + padding+1: shiftx + padding+W] = gsub;

padz = zeros((H + 2*padding, W + 2*padding));

In [ ]:
colorview(RGB, padz, padr, padg)

Repeat that process for the "i" band!

In [ ]:
I,J = find_nearby(rx,ry, ix,iy, 50.)
dx = rx[I] - ix[J]
dy = ry[I] - iy[J];
six,siy = find_most_common(dx, dy)

In [ ]:
padi = zeros((H + 2*padding, W + 2*padding))
padi[siy + padding+1 : siy + padding+H, six + padding+1: six + padding+W] = isub;

In [ ]:
colorview(RGB, padi, padr, padg)

We can use different "stretch" functions to make pretty pictures.

In [ ]:
offset = 0.1
rgb = colorview(RGB, offset .+ padi, offset .+ padr.*1.5, offset .+ padg.*2.)

In [ ]:
rgb = colorview(RGB,
    sqrt.(clamp.(padi,      0, 1)),
    sqrt.(clamp.(padr.*1.5, 0, 1)),
    sqrt.(clamp.(padg.*2.,  0, 1)))